In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("/content/drive/MyDrive/PAYTM_historical_data.csv")
data


,Date,Open,High,Low,Close,Volume
0,2022-08-01T09:15:00+05:30,708.00,713.90,707.45,712.75,47041
1,2022-08-01T09:16:00+05:30,713.40,714.00,710.30,711.90,19707
2,2022-08-01T09:17:00+05:30,711.05,712.10,711.00,711.30,14889
3,2022-08-01T09:18:00+05:30,712.00,712.65,711.20,711.95,11883
4,2022-08-01T09:19:00+05:30,712.35,713.55,712.15,712.55,14532
...,...,...,...,...,...,...
184495,2023-07-31T15:25:00+05:30,644.80,644.80,644.15,644.15,19116
184496,2023-07-31T15:26:00+05:30,644.35,644.70,644.15,644.70,22111
184497,2023-07-31T15:27:00+05:30,644.70,644.70,644.35,644.40,24553
184498,2023-07-31T15:28:00+05:30,644.40,644.65,642.90,644.00,67930


In [3]:
# Define strategy functions

def moving_average_crossover(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Calculate moving averages (adjust window sizes as needed)
    short_window = 50
    long_window = 200
    data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window).mean()

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(1, len(data)):
        if data['Short_MA'][i] > data['Long_MA'][i] and data['Short_MA'][i - 1] <= data['Long_MA'][i - 1]:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if data['Short_MA'][i] < data['Long_MA'][i] and data['Short_MA'][i - 1] >= data['Long_MA'][i - 1]:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def bollinger_bands(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Calculate Bollinger Bands
    window = 20
    data['Rolling_Mean'] = data['Close'].rolling(window=window).mean()
    data['Rolling_Std'] = data['Close'].rolling(window=window).std()
    data['Upper_Band'] = data['Rolling_Mean'] + 2 * data['Rolling_Std']
    data['Lower_Band'] = data['Rolling_Mean'] - 2 * data['Rolling_Std']

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(window, len(data)):
        if close_prices[i] > data['Upper_Band'][i] and close_prices[i - 1] <= data['Upper_Band'][i - 1]:
            if breakout_sell_signal == 1:
                position = "Sell"
                entry_price = close_prices[i]

        if close_prices[i] < data['Lower_Band'][i] and close_prices[i - 1] >= data['Lower_Band'][i - 1]:
            if breakout_buy_signal == 1 and position == "Sell":
                position = None
                exit_price = close_prices[i]
                profit += (entry_price - exit_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def rsi_mean_reversion(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices
    rsi_period = 14  # RSI calculation period

    # Calculate RSI
    delta = np.diff(close_prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = np.zeros_like(close_prices)
    avg_loss = np.zeros_like(close_prices)
    avg_gain[rsi_period] = np.mean(gain[:rsi_period])
    avg_loss[rsi_period] = np.mean(loss[:rsi_period])
    for i in range(rsi_period + 1, len(close_prices)):
        avg_gain[i] = (avg_gain[i - 1] * (rsi_period - 1) + gain[i - 1]) / rsi_period
        avg_loss[i] = (avg_loss[i - 1] * (rsi_period - 1) + loss[i - 1]) / rsi_period
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(rsi_period, len(data)):
        if rsi[i] > 70 and rsi[i - 1] <= 70:
            if breakout_sell_signal == 1:
                position = "Sell"
                entry_price = close_prices[i]

        if rsi[i] < 30 and rsi[i - 1] >= 30:
            if breakout_buy_signal == 1 and position == "Sell":
                position = None
                exit_price = close_prices[i]
                profit += (entry_price - exit_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100
    return profit, profit_percentage

def macd_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Calculate MACD (adjust parameters as needed)
    short_window = 12
    long_window = 26
    signal_window = 9
    data['Short_EMA'] = data['Close'].ewm(span=short_window, adjust=False).mean()
    data['Long_EMA'] = data['Close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['Short_EMA'] - data['Long_EMA']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(1, len(data)):
        if data['MACD'][i] > data['Signal_Line'][i] and data['MACD'][i - 1] <= data['Signal_Line'][i - 1]:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if data['MACD'][i] < data['Signal_Line'][i] and data['MACD'][i - 1] >= data['Signal_Line'][i - 1]:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100
    return profit, profit_percentage

def commodity_channel_index(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices
    high_prices = data['High'].tolist()  # Assuming 'High' column contains the high prices
    low_prices = data['Low'].tolist()  # Assuming 'Low' column contains the low prices
    period = 20  # CCI calculation period

    # Calculate Typical Price and Simple Moving Average of Typical Price
    typical_prices = (np.array(high_prices) + np.array(low_prices) + np.array(close_prices)) / 3
    sma_typical_prices = np.convolve(typical_prices, np.ones(period)/period, mode='valid')

    # Calculate Mean Deviation
    mean_deviation = []
    for i in range(period, len(data)):
        deviation = np.abs(typical_prices[i - period + 1:i + 1] - sma_typical_prices[i - period])
        mean_deviation.append(np.mean(deviation))

    # Calculate Commodity Channel Index (CCI)
    cci = (typical_prices[period - 1:] - sma_typical_prices) / (0.015 * np.array(mean_deviation))

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(period, len(data)):
        if cci[i - period] > 100 and cci[i - period - 1] <= 100:
            if breakout_sell_signal == 1:
                position = "Sell"
                entry_price = close_prices[i]

        if cci[i - period] < -100 and cci[i - period - 1] >= -100:
            if breakout_buy_signal == 1 and position == "Sell":
                position = None
                exit_price = close_prices[i]
                profit += (entry_price - exit_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100
    return profit, profit_percentage

def  average_directional_index(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    high_prices = data['High'].tolist()  # Assuming 'High' column contains the high prices
    low_prices = data['Low'].tolist()  # Assuming 'Low' column contains the low prices
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the close prices
    period = 14  # ADX calculation period

    # Calculate True Range (TR)
    tr_list = []
    for i in range(1, len(data)):
        tr = max(high_prices[i] - low_prices[i], abs(high_prices[i] - close_prices[i - 1]), abs(low_prices[i] - close_prices[i - 1]))
        tr_list.append(tr)
    atr = np.mean(tr_list[-period:])

    # Calculate Directional Movement
    dm_plus = []
    dm_minus = []
    for i in range(1, len(data)):
        dm_plus.append(max(high_prices[i] - high_prices[i - 1], 0))
        dm_minus.append(max(low_prices[i - 1] - low_prices[i], 0))
    atr_dm_plus = np.mean(dm_plus[-period:])
    atr_dm_minus = np.mean(dm_minus[-period:])

    # Calculate Directional Index (DI+ and DI-)
    di_plus = (atr_dm_plus / atr) * 100
    di_minus = (atr_dm_minus / atr) * 100

    # Calculate Average Directional Index (ADX)
    dx = abs((di_plus - di_minus) / (di_plus + di_minus)) * 100
    adx = np.mean(dx[-period:])

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(period, len(data)):
        if adx[i - period] > 25 and adx[i - period - 1] <= 25:
            if breakout_sell_signal == 1:
                position = "Sell"
                entry_price = close_prices[i]

        if adx[i - period] < 20 and adx[i - period - 1] >= 20:
            if breakout_buy_signal == 1 and position == "Sell":
                position = None
                exit_price = close_prices[i]
                profit += (entry_price - exit_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100
    return profit, profit_percentage

def On_Balance_Volume_Indicator(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices
    volume = data['Volume'].tolist()  # Assuming 'Volume' column contains the trading volume

    # Calculate OBV
    obv = [0]
    for i in range(1, len(data)):
        if close_prices[i] > close_prices[i - 1]:
            obv.append(obv[i - 1] + volume[i])
        elif close_prices[i] < close_prices[i - 1]:
            obv.append(obv[i - 1] - volume[i])
        else:
            obv.append(obv[i - 1])

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(1, len(data)):
        if obv[i] > obv[i - 1] and obv[i - 1] <= 0:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if obv[i] < obv[i - 1] and obv[i - 1] >= 0:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100
    return profit, profit_percentage


def average_true_range(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the close prices
    period = 14  # ATR calculation period

    # Calculate True Range (TR)
    tr_list = [high - low for high, low in zip(data['High'], data['Low'])]

    # Calculate Average True Range (ATR)
    atr = [sum(tr_list[:period]) / period]
    for i in range(period, len(tr_list)):
        atr.append((atr[-1] * (period - 1) + tr_list[i]) / period)

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(period, len(data)):
        if close_prices[i] > close_prices[i - 1] + atr[i - period] and close_prices[i - 1] <= close_prices[i - 2] + atr[i - period - 1]:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if close_prices[i] < close_prices[i - 1] - atr[i - period] and close_prices[i - 1] >= close_prices[i - 2] - atr[i - period - 1]:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100
    return profit, profit_percentage

def Stochastic_Oscillator(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the close prices
    period = 14  # Stochastic Oscillator calculation period

    # Calculate %K Stochastic Oscillator
    stochastic_k = []
    for i in range(period, len(data)):
        highest_high = max(close_prices[i - period + 1:i + 1])
        lowest_low = min(close_prices[i - period + 1:i + 1])
        k = ((close_prices[i] - lowest_low) / (highest_high - lowest_low)) * 100
        stochastic_k.append(k)

    # Calculate %D Stochastic Oscillator
    stochastic_d = [sum(stochastic_k[:period]) / period]
    for i in range(period, len(stochastic_k)):
        stochastic_d.append((stochastic_d[-1] * (period - 1) + stochastic_k[i]) / period)

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(period, len(data)):
        if stochastic_k[i - period] < 20 and stochastic_d[i - period] < 20 and stochastic_k[i - period - 1] >= 20 and stochastic_d[i - period - 1] >= 20:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if stochastic_k[i - period] > 80 and stochastic_d[i - period] > 80 and stochastic_k[i - period - 1] <= 80 and stochastic_d[i - period - 1] <= 80:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage


# Map strategy names to their corresponding functions
strategy_functions = {
    "moving average crossover":moving_average_crossover ,
    "bollinger bands": bollinger_bands,
    "rsi mean reversion":rsi_mean_reversion,
    "macd strategy":macd_strategy,
    "commodity channel index":commodity_channel_index,
    "average directional index":average_directional_index,
    "obv Indicator":On_Balance_Volume_Indicator,
    "average true range":average_true_range,
    "Stochastic Oscillator":Stochastic_Oscillator,
}

# Get user inputs
strategy_name = input("Enter the name of the strategy\n(moving average crossover,\nbollinger bands,\nrsi mean reversion,\nmacd strategy,\ncommodity channel index,\naverage directional index,\nobv Indicator,\naverage true range,\nStochastic Oscillator): ")
num_shares = int(input("Enter the number of shares: "))
breakout_buy_signal = int(input("Enter the value(0 or 1) for Breakout_Buy_Signal: "))
breakout_sell_signal = int(input("Enter the value(0 or 1) for Breakout_Sell_Signal: "))

# Check if the selected strategy is in the dictionary
if strategy_name in strategy_functions:
    # Call the corresponding strategy function to calculate profit
    calculate_profit_function = strategy_functions[strategy_name]
    profit, profit_percentage = calculate_profit_function(num_shares, data, breakout_buy_signal, breakout_sell_signal)

    # Print results
    print(f"Profit: {profit:.2f}")
    print(f"Profit Percentage: {profit_percentage:.2f}%")
else:
    print(f"Strategy '{strategy_name}' not found.")


Enter the name of the strategy
(moving average crossover,
bollinger bands,
rsi mean reversion,
macd strategy,
commodity channel index,
average directional index,
obv Indicator,
average true range,
Stochastic Oscillator): macd strategy
Enter the number of shares: 1
Enter the value(0 or 1) for Breakout_Buy_Signal: 1
Enter the value(0 or 1) for Breakout_Sell_Signal: 1
Profit: -418.75
Profit Percentage: -64.95%
